### Install pyenv on WSL

```
# Clone pyenv
curl https://pyenv.run | bash
# Add to your shell startup (e.g. ~/.bashrc or ~/.zshrc):
export PATH="$HOME/.pyenv/bin:$PATH"
eval "$(pyenv init --path)"
eval "$(pyenv init -)"
eval "$(pyenv virtualenv-init -)"
# Then reload:
exec $SHELL
```

### Build Python 3.10.12

```
pyenv install 3.10.12
# Verify:
pyenv versions
```

### Create & activate a pyenv-venv

```
pyenv virtualenv 3.10.12 yakuza_venv
pyenv activate yakuza_venv
```

### Install Jupyter & ipykernel to work with Jypyter Notebooks

```
pip install jupyter ipykernel
```

### Install necessary libraries

```
pip install numpy pandas torch Bio transformers fair-esm xgboost scikit-learn
```

### Download Serratia bacterial genomes from NCBI

```
pip install ncbi-genome-download
ncbi-genome-download \
    --section genbank \
    --formats fasta \
    --assembly-levels complete \
    --species-taxid 615,82996 \
    --output-folder ./data/ncbi_bacteria \
    bacteria
```

### pharokka installation
#### Using pip:
```
pip install pharokka PHANOTATE
git clone https://github.com/gbouras13/pharokka.git
```

#### Alternatively with conda:
```
sudo apt install -y wget git build-essential hmmer barrnap trnascan-se
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh
```
#### Restart terminal to finalize conda installation
```
conda env create -f pharokka/environment.yml
conda activate pharokka_env
```

### Install pharokka databases:
```
./pharokka/bin/install_databases.py
```

#### MMseqs2:
```
wget https://github.com/soedinglab/MMseqs2/releases/download/13-45111/mmseqs-linux-avx2.tar.gz
tar xvfz mmseqs-linux-avx2.tar.gz
```

#### MinCED:
```
sudo apt install openjdk-11-jre
sudo apt install openjdk-11-jdk-headless
git clone https://github.com/ctSkennerton/minced.git
cd minced
make
sudo cp minced.jar /usr/local/bin/
sudo cp minced /usr/local/bin/
cd ..
```

#### ARAGORN:
```
mkdir aragorn; cd aragorn
wget https://anaconda.org/bioconda/aragorn/1.2.41/download/linux-64/aragorn-1.2.41-h031d066_3.tar.bz2
tar -xvjf aragorn-1.2.41-h031d066_3.tar.bz2
sudo cp bin/aragorn /usr/local/bin/
cd ..
```

#### Mash:
```
wget https://github.com/marbl/Mash/releases/download/v2.3/mash-Linux64-v2.3.tar
tar -xvf mash-Linux64-v2.3.tar
sudo cp mash-Linux64-v2.3/mash /usr/local/bin/
```

#### Dnaapler:
```
pip3 install dnaapler
```

### Install RBPdetect_v4_ESMfinetuned

```
sudo apt install unzip
wget 'https://zenodo.org/records/14810759/files/RBPdetect_v4_ESMfine.zip?download=1' -O 'RBPdetect_v4_ESMfine.zip'
unzip ./RBPdetect_v4_ESMfine.zip -d .
rm -f RBPdetect_v4_ESMfine.zip
```

### Bakta
#### Using conda (recommended):
```
conda install -c conda-forge -c bioconda bakta
```

#### Using pip (must install 3rd party dependencies manually):
```
pip install bakta
```

### AMRFinderPlus:
```
mkdir AMRFinderPlus; cd AMRFinderPlus
wget https://github.com/ncbi/amr/releases/download/amrfinder_v4.0.3/amrfinder_binaries_v4.0.3.tar.gz
tar -xvzf amrfinder_binaries_v4.0.3.tar.gz
cd ..
sudo cp ./AMRFinderPlus/* /usr/local/bin/
```

### PILER-CR:
```
wget http://www.drive5.com/pilercr/pilercr1.06.tar.gz
tar -xvzf pilercr1.06.tar.gz
cd pilercr1.06
make
sudo cp ./pilercr /usr/local/bin/
cd ..
```

### Diamond:
```
wget https://github.com/bbuchfink/diamond/releases/download/v2.1.10/diamond-linux64.tar.gz
tar -xvzf diamond-linux64.tar.gz
sudo mv diamond /usr/local/bin/
```

### Blast+ (update from 2.12.0 to 2.14.0):
```
sudo apt remove -y ncbi-blast+
wget https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/2.14.0/ncbi-blast-2.14.0+-x64-linux.tar.gz
tar -xvzf ncbi-blast-2.14.0+-x64-linux.tar.gz
sudo mv ncbi-blast-2.14.0+/bin/* /usr/local/bin/
```

## Install full database (~30GB):
```
mkdir bakta_db
bakta_db download --output ./bakta_db --type full
```

### Run against a single genome .fasta:
```
mkdir ./data/bakta_test
bakta --db ./bakta_db/db --output ./data/bakta_test --force ./data/bacteria_genomes/33KP-HG.fasta
bakta --db ./bakta_db/db \
      --output ./data/bakta_test \
      --force \
      --skip-trna \
      --skip-tmrna \
      --skip-rrna \
      --skip-ncrna \
      --skip-ncrna-region \
      --skip-crispr \
      --skip-pseudo \
      --skip-sorf \
      --skip-gap \
      --skip-plot \
      ./data/bacteria_genomes/33KP-HG.fasta
```

### In general:
#### For training:
```
bakta --db ./bakta_db/db \
      --output <bakta_output_path> \
      --force \
      --skip-trna \
      --skip-tmrna \
      --skip-rrna \
      --skip-ncrna \
      --skip-ncrna-region \
      --skip-crispr \
      --skip-pseudo \
      --skip-sorf \
      --skip-gap \
      --skip-plot \
      <serratia_genome>.fasta
```
#### For inference:
```
bakta --db <db_path> \
      --output <bakta_output_path> \
      --force \
      <serratia_genome>.fasta
```

### Ray Tune

```
pip install ray[tune]
```

In [ ]:
# 1 - INITIAL SETUP
# --------------------------------------------------
import numpy as np
import pickle
from tqdm import tqdm
from pathlib import Path
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from functools import partial
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score

import serraphim_processing as sp
import serraphim_features as sf

# Project path
main_path = '/home/snt/SerraPHIM'

# General paths
code_path = main_path + '/code'
data_path = main_path + '/data'
results_path = main_path + '/results'

# Bacterial genome data paths
bact_genomes_path = data_path + '/bacteria_genomes'
ncbi_bact_path = data_path + '/ncbi_bacteria/genbank/bacteria'

# Phage genome data and metadata paths
phage_genomes_path = data_path + '/phage_genomes'
phagescope_multiFASTA_path = data_path + '/PhageScope_data/PhageScope_virulent_complete-HQ_sequences.fasta'

phagescope_metadata_path = data_path + '/PhageScope_metadata'
metadata_file = 'serratia_complete-HQ_virulent_phage_metadata.tsv'

data_suffix = '_training'

# Software and database paths
phl_path = '/home/snt/PhageHostLearn-main'
pharokka_executable_path = phl_path + '/pharokka/bin/pharokka.py'
pharokka_db_path = phl_path + '/pharokka/databases'

RBP_detect_model_path = main_path + '/RBPdetect_v4_ESMfine'

bakta_db_path = phl_path + '/bakta_db/db'
bakta_results_path = data_path + '/bakta_results'

# Paths for testing purposes:
phage_genomes_sample_path = phage_genomes_path + '/sample'
bact_genomes_sample_path = bact_genomes_path + '/sample'

In [2]:
# 2 - DATA PROCESSING
# --------------------------------------------------

# Check if the bacterial genomes' directory exists and is empty
if not Path(bact_genomes_path).exists() or not any(Path(bact_genomes_path).iterdir()):
    print("Bacterial genomes' directory is empty. Processing NCBI FASTA files...")
    sp.process_ncbi_fasta(
        input_root=ncbi_bact_path,
        output_root=bact_genomes_path,
        combine_plasmids="True"
    )
    # combine_plasmids="True" -> 284 FASTA files saved (plasmid-only FNA files skipped)
    # combine_plasmids="False" -> 578 FASTA files saved
else:
    print("Bacterial genomes' directory is not empty. Moving on...")
    pass

Bacterial genomes' directory is not empty. Moving on...


In [3]:
# Create or load phage metadata
phage_metadata = sp.load_or_create_curated_phage_metadata(
    phagescope_metadata_path,
    metadata_file
)

# Split the multi-FASTA file manually created using PhageScope
if not Path(phage_genomes_path).exists() or not any(Path(phage_genomes_path).iterdir()):
    print("Phage genomes' directory is empty. Processing PhageScope multi-FASTA file...")
    sp.split_fasta_by_phage_id(
        phagescope_multiFASTA_path,
        phage_genomes_path
    )
else:
    print("Phage genomes' directory is not empty. Moving on...")
    pass

Metadata file already exists: serratia_complete-HQ_virulent_phage_metadata.tsv
Phage genomes' directory is not empty. Moving on...


In [4]:
# Derive interaction matrix
interaction_filepath = data_path + f'/interactions/phage_host_interactions{data_suffix}.csv'

phage_host_interaction_matrix = sp.derive_phage_host_interaction_matrix(
    bact_genomes_path,
    phage_metadata,
    interaction_filepath
)

Interaction matrix already exists at: /home/snt/SerraPHIM/data/interactions/phage_host_interactions_training.csv


In [ ]:
# Run pharokka
sp.pharokka_processing(
    data_path, 
    phage_genomes_path, # phage_genomes_path or phage_genomes_sample_path
    pharokka_executable_path, 
    pharokka_db_path, 
    data_suffix,
    threads=16  # 128 cores in yakuza server
)

  0%|          | 0/5 [00:00<?, ?it/s]

/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:558: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:90: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:558: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:90: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
2025-07-25 15:51:20.958 | INFO     | i

 20%|██        | 1/5 [02:28<09:54, 148.59s/it]

/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:558: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:90: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:558: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:90: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
2025-07-25 15:53:49.243 | INFO     | i

 40%|████      | 2/5 [05:04<07:38, 152.94s/it]

...

/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:558: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:90: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:558: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:90: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
2025-07-25 15:59:09.284 | INFO     | i

 80%|████████  | 4/5 [10:36<02:41, 161.96s/it]

/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:558: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:90: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:558: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/numpy/_core/getlimits.py:90: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
2025-07-25 16:01:57.417 | INFO     | i

100%|██████████| 5/5 [14:23<00:00, 172.77s/it]

Pharokka processing complete.


In [ ]:
# Run RBP detection using PhageRBPdetect_v4
# WARNING: No GPUs in yakuza server!
sp.process_and_detect_rbps_v4(
    data_path, 
    RBP_detect_model_path, 
    data_suffix,
    gpu_index=0,
    threads=16  # 128 cores in yakuza server
)

Using device: cuda


Inference for uvig_32658: 100%|██████████| 18/18 [00:00<00:00, 26.70it/s]
Inference for IMGVR_UViG_640753048_000002_640753048_640753089: 100%|██████████| 93/93 [00:01<00:00, 69.34it/s]
Inference for NC_020083.1: 100%|██████████| 228/228 [00:03<00:00, 63.66it/s]
Inference for IMGVR_UViG_3300031415_000018_3300031415_Ga0308172_1000631: 100%|██████████| 74/74 [00:01<00:00, 69.43it/s]
Processing phages: 100%|██████████| 5/5 [00:07<00:00,  1.56s/it]

RBP detection complete. Results saved to 'RBPbase_training.csv'


In [7]:
# Run Bakta
product_keywords = [
    "capsule", "polysaccharide", "-polysaccharide", "LPS",
    "oligosaccharide polymerase",
    "UDP-", "GDP-",
    "mannose",
    "guanylyltransferase",
    "glycosyltransferase",
    "flippase",
    "-antigen", "O-antigen",
    "outer membrane", "outer core", "omp-",
    "mannosyltransferase",
    "fimbria-",
    "pili-", "pilus",
    "adhesin",
    "cellulose",
    "curli",
    # "receptor", # too generic? any receptors not related to surface receptors? 
    "siderophore receptor", "siderophore",
    "enterobactin receptor",
    "aerobactin receptor",
    "yersiniabactin receptor",
    "salmochelin receptor",
    "phage receptor",
    # "prophage", # how is a possible prophage existence affecting the susceptibility to other phages?
    # "phage", "tail fiber", # + other phage components... (CDS in prophage regions?)
    "porin", "-porin",
    "efflux pump",
    "hemagglutinin", "haemagglutinin",
    "lectin",
    "lipid", "-lipid", 
    # "lipoprotein", # too generic? any lipoproteins not related to cell wall / surface receptors?
    "beta-barrel assembly",
    "autotransporter",
    "pathway protein",
    "flagell-",
    "chemotaxis",
    "transporter membrane", "transmembrane",
    "TonB-dependent receptor", "TonB", "TonB-",
    "chitinase", "lecithinase", "hemolysin", 
    "lipase", "-lipase", 
    "protease", "-protease",
    "exoenzyme-", "virulence", "invasion"
]
gene_keywords = [
    "wza", # (capsule export)
    "wzb", # (tyrosine-protein phosphatase)
    "wzc", # (capsule synthesis tyrosine kinase)
    "galF", # (UDP-glucose pyrophosphorylase)
    "ugd", # (UDP-glucose dehydrogenase)
    "manC", # (GDP-mannose pyrophosphorylase)
    "manB", # (mannose-1-phosphate guanylyltransferase)
    "Wzx", # (O-antigen flippase)
    "Wzy", # (O-antigen polymerase)
    "WaaL", # (O-antigen ligase)
    "Waa-", # (core LPS biosynthesis)
    #"magA", # (K1-specific capsular serotype proteins)
    #"rmpA", # (K2-specific capsular serotype proteins)
    "OmpA", "OmpX", "Omp-", # (outer membrane proteins)
    #"OmpK-", # (Klebsiella-specific outer membrane proteins)
    "TolC", # (outer membrane proteins)
    "OprD", "Opr-", # (outer membrane proteins)
    "HasR", "HasB", # (TonB-dependent outer membrane receptor complex)
    "lpx-", # (lipid A biosynthesis: lpxA, lpxC, lpxD)
    "WbbP", # (mannosyltransferase)
    "fimA", "fimH", # (Type 1 fimbriae)
    "mrkA", "mrkB", "mrkC", # (Type 3 fimbriae)
    "PilQ", "PilX", # (Adhesins)
    "csgA", "csgB", # (Curli fibers)
    "FliD", # (Flagellar hook-associated proteins)
    "CheA", "CheB", "CheR", "CheY", "CheW", "CheZ", "Che-", # (Chemotaxis proteins)
    "FepA", "Fiu", # (Enterobactin receptors)
    "IutA", # (Aerobactin receptors)
    "FyuA", # (Yersiniabactin receptors)
    "IroN" # (Salmochelin receptors)
]
sp.run_bakta_and_extract_receptors(
    bact_genomes_sample_path, # bact_genomes_path, 
    bakta_results_path, 
    bakta_db_path, 
    gene_keywords, product_keywords,
    data_suffix,
    threads=16,  # 128 cores in yakuza server
    training=True
)

Processing genomes:   0%|          | 0/5 [00:00<?, ?it/s]

Command: bakta --db /home/snt/PhageHostLearn-main/bakta_db/db --output /home/snt/SerraPHIM/data/bakta_results/CP100764.1_Serratia_plymuthica_strain_SASK2000_chromosome --force --threads 16 --skip-trna --skip-tmrna --skip-rrna --skip-ncrna --skip-ncrna-region --skip-crispr --skip-pseudo --skip-sorf --skip-gap --skip-plot /home/snt/SerraPHIM/data/bacteria_genomes/sample/CP100764.1_Serratia_plymuthica_strain_SASK2000_chromosome.fasta
Parse genome sequences...
	imported: 1
	filtered & revised: 1
	chromosomes: 1

Start annotation...
skip tRNA prediction...
skip tmRNA prediction...
skip rRNA prediction...
skip ncRNA prediction...
skip ncRNA region prediction...
skip CRISPR array prediction...
predict & annotate CDSs...
	predicted: 5041 
	discarded spurious: 1
	revised translational exceptions: 0
	detected IPSs: 4764
	found PSCs: 138
	found PSCCs: 106
	lookup annotations...
	conduct expert systems...
		amrfinder: 7
		protein sequences: 50
	combine annotations and mark hypotheticals...
	analyz

Processing genomes:  20%|██        | 1/5 [03:05<12:22, 185.57s/it]

Receptor genes:      Sequence Id Type    Start     Stop Strand     Locus Tag  Gene  \
21      contig_1  cds    24080    24739      -  EFMCOH_00022  ompA   
24      contig_1  cds    26068    28056      +  EFMCOH_00025   NaN   
70      contig_1  cds    77521    78864      +  EFMCOH_00071   NaN   
89      contig_1  cds    98711   100105      +  EFMCOH_00090   NaN   
93      contig_1  cds   102091   103785      +  EFMCOH_00094  eptB   
...          ...  ...      ...      ...    ...           ...   ...   
4977    contig_1  cds  5369170  5370060      -  EFMCOH_04978  brkB   
4991    contig_1  cds  5387832  5388425      +  EFMCOH_04992  mobA   
5019    contig_1  cds  5420906  5422276      +  EFMCOH_05019  glmU   
5028    contig_1  cds  5430606  5431319      +  EFMCOH_05028  hisM   
5032    contig_1  cds  5434333  5435382      -  EFMCOH_05032   NaN   

                                                Product  \
21                              OmpA family lipoprotein   
24                       

Processing genomes:  40%|████      | 2/5 [06:09<09:13, 184.50s/it]

...

Receptor genes:      Sequence Id Type  Start   Stop Strand     Locus Tag  Gene  \
39      contig_1  cds  46272  48626      -  LCBGCM_00040  lptD   
68      contig_1  cds  85151  86638      +  LCBGCM_00069  murE   
69      contig_1  cds  86635  87996      +  LCBGCM_00070  murF   
71      contig_1  cds  89075  90394      +  LCBGCM_00072  murD   
74      contig_1  cds  92746  94221      +  LCBGCM_00075  murC   
...          ...  ...    ...    ...    ...           ...   ...   
4862    contig_3  cds  24527  24880      +  LCBGCM_04861  traA   
4872    contig_3  cds  33715  34584      +  LCBGCM_04871  traU   
4873    contig_3  cds  34590  34985      +  LCBGCM_04872  trbC   
4875    contig_3  cds  36950  37720      +  LCBGCM_04874  traF   
4885    contig_3  cds  53145  53660      +  LCBGCM_04884   NaN   

                                                Product  \
39                            LPS assembly protein LptD   
68    UDP-N-acetylmuramoyl-L-alanyl-D-glutamate--2,6...   
69    UDP-N-ac

Processing genomes:  80%|████████  | 4/5 [12:37<03:12, 192.56s/it]

Receptor genes:      Sequence Id Type    Start     Stop Strand     Locus Tag  Gene  \
39      contig_1  cds    46257    48617      -  OANAOL_00040  lptD   
68      contig_1  cds    85142    86629      +  OANAOL_00069  murE   
69      contig_1  cds    86626    87987      +  OANAOL_00070  murF   
71      contig_1  cds    89066    90385      +  OANAOL_00072  murD   
74      contig_1  cds    92737    94212      +  OANAOL_00075  murC   
...          ...  ...      ...      ...    ...           ...   ...   
4955    contig_1  cds  5391145  5392347      -  OANAOL_04954  araJ   
4983    contig_1  cds  5419174  5420607      +  OANAOL_04982   NaN   
4987    contig_1  cds  5423699  5424667      -  OANAOL_04986   NaN   
4990    contig_1  cds  5427576  5428073      -  OANAOL_04989  ompA   
4997    contig_1  cds  5434195  5435064      -  OANAOL_04996  robA   

                                                Product  \
39                            LPS assembly protein LptD   
68    UDP-N-acetylmuramoy

Processing genomes: 100%|██████████| 5/5 [15:56<00:00, 191.26s/it]

Receptor genes:      Sequence Id Type  Start    Stop Strand     Locus Tag  Gene  \
21      contig_1  cds  23947   24606      -  EPGMHD_00022  ompA   
24      contig_1  cds  25935   27923      +  EPGMHD_00025   NaN   
67      contig_1  cds  76026   76334      +  EPGMHD_00068  rhuM   
85      contig_1  cds  96510   97904      +  EPGMHD_00086   NaN   
89      contig_1  cds  99891  101585      +  EPGMHD_00090  eptB   
...          ...  ...    ...     ...    ...           ...   ...   
4879    contig_2  cds  31873   32868      +  EPGMHD_04879  traU   
4880    contig_2  cds  32880   33485      +  EPGMHD_04880  trbC   
4882    contig_2  cds  35503   36288      +  EPGMHD_04882  traF   
4887    contig_2  cds  38412   39791      +  EPGMHD_04887  traH   
4893    contig_2  cds  52066   52605      +  EPGMHD_04893   NaN   

                                                Product  \
21                              OmpA family lipoprotein   
24                                             Lipase 1   
67

In [8]:
# 3 - FEATURE CONSTRUCTION
# --------------------------------------------------

# ESM-2 features for RBPs
sf.compute_esm2_embeddings_rbp_improved(
    data_path, 
    data_suffix, 
    batch_size=16
)

Embedding sequences: 100%|██████████| 14/14 [01:00<00:00,  4.32s/it]

Saved embeddings to /home/snt/SerraPHIM/data/esm2_embeddings_rbp_training.csv


In [9]:
# ESM-2 features for bacterial receptors 
sf.compute_esm2_embeddings_receptors(
    data_path, 
    bakta_results_path, 
    data_suffix,
    aggregate_by_accession=True,  # Set to true during internship
    batch_size=16,
    skip_long_seq=False
)

Will process CP100764.1_Serratia_plymuthica_strain_SASK2000_chromosome_288 solo (length=1880 AA)
Will process CP100764.1_Serratia_plymuthica_strain_SASK2000_chromosome_320 solo (length=2154 AA)
Will process CP100764.1_Serratia_plymuthica_strain_SASK2000_chromosome_342 solo (length=3314 AA)
Will process CP100764.1_Serratia_plymuthica_strain_SASK2000_chromosome_349 solo (length=1607 AA)
Will process OX291654.1_Serratia_marcescens_strain_SJC1039_genome_assembly_92 solo (length=3548 AA)
Will process OX291654.1_Serratia_marcescens_strain_SJC1039_genome_assembly_275 solo (length=1608 AA)
Will process AP013063.1_Serratia_marcescens_SM39_DNA_+_plasmids_141 solo (length=3642 AA)
Will process AP013063.1_Serratia_marcescens_SM39_DNA_+_plasmids_324 solo (length=1608 AA)
Will process AP028466.1_Serratia_marcescens_12BL1_DNA_147 solo (length=2722 AA)
Will process AP028466.1_Serratia_marcescens_12BL1_DNA_271 solo (length=2023 AA)
Will process AP028466.1_Serratia_marcescens_12BL1_DNA_338 solo (length=

Embedding batches: 100%|██████████| 14/14 [03:05<00:00, 13.27s/it]


Saved embeddings to: /home/snt/SerraPHIM/data/esm2_embeddings_receptors_training.csv


In [10]:
# Construct feature matrices
rbp_embeddings_path = data_path + '/esm2_embeddings_rbp' + data_suffix + '.csv'
receptors_embeddings_path = data_path + '/esm2_embeddings_receptors' + data_suffix + '.csv'
interaction_path = data_path + '/interactions'

features_esm2, labels, groups_receptors, groups_phage = sf.construct_feature_matrices_receptors(
    interaction_path, 
    data_suffix, 
    receptors_embeddings_path, 
    rbp_embeddings_path
)

Building feature matrix: 100%|██████████| 20/20 [00:00<00:00, 1008.08it/s]


In [11]:
features_esm2, labels, groups_receptors, groups_phage

(array([[np.float64(-0.014332249), np.float64(-0.06588216),
         np.float64(-0.0023698194), ..., np.float64(-0.071024075),
         np.float64(-0.013574367666666665),
         np.float64(0.10997433999999999)],
        [np.float64(-0.014332249), np.float64(-0.06588216),
         np.float64(-0.0023698194), ..., np.float64(-0.12421417),
         np.float64(-0.0037242933), np.float64(0.1801741)],
        [np.float64(-0.014332249), np.float64(-0.06588216),
         np.float64(-0.0023698194), ..., np.float64(-0.07182163700000001),
         np.float64(0.0048919260000000004), np.float64(0.0896083225)],
        ...,
        [np.float64(-0.015825137), np.float64(-0.06701858),
         np.float64(-0.001265623), ..., np.float64(-0.12421417),
         np.float64(-0.0037242933), np.float64(0.1801741)],
        [np.float64(-0.015825137), np.float64(-0.06701858),
         np.float64(-0.001265623), ..., np.float64(-0.07182163700000001),
         np.float64(0.0048919260000000004), np.float64(0.08960

In [ ]:
# 4 - TRAINING & EVALUATING MODELS
# --------------------------------------------------

X_full = features_esm2
y_full = labels
groups_receptors = np.array(groups_receptors)
groups_phage = np.array(groups_phage)

imbalance = np.sum(y_full == 1) / np.sum(y_full == 0)

/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [18:13:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [ ]:
search_space = {
    "learning_rate": tune.loguniform(1e-3, 0.3),
    "max_depth": tune.choice([3, 5, 7, 9]),
    "n_estimators": tune.choice([100, 200, 300]),
    "scale_pos_weight": tune.choice(1 / imbalance),
    "subsample": tune.uniform(0.6, 1.0),
    "colsample_bytree": tune.uniform(0.6, 1.0)
}

def train_xgb_tune(config, X, y):
    # Stratified train/test split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    model = XGBClassifier(
        learning_rate=config["learning_rate"],
        max_depth=config["max_depth"],
        n_estimators=config["n_estimators"],
        scale_pos_weight=config["scale_pos_weight"],
        subsample=config["subsample"],
        colsample_bytree=config["colsample_bytree"],
        eval_metric="logloss",
        use_label_encoder=False,
        n_jobs=8
    )
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)
    
    tune.report({"auc": auc})

In [29]:
analysis = tune.run(
    partial(train_xgb_tune, X=X_full, y=y_full),
    config=search_space,
    num_samples=30,  # increase for more exhaustive search
    metric="auc",
    mode="max",
    scheduler=ASHAScheduler(max_t=50, grace_period=5),
    resources_per_trial={"cpu": 8},
    storage_path=f"{data_path}/ray_tune_results",
    name="xgb_receptor_tune"
)

2025-07-25 19:52:21,600	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,auc
train_xgb_tune_1d5bc_00000,0.75
train_xgb_tune_1d5bc_00001,0.75
train_xgb_tune_1d5bc_00002,0.75
train_xgb_tune_1d5bc_00003,0.5
train_xgb_tune_1d5bc_00004,1
train_xgb_tune_1d5bc_00005,0.75
train_xgb_tune_1d5bc_00006,0.75
train_xgb_tune_1d5bc_00007,0.75
train_xgb_tune_1d5bc_00008,0.75
train_xgb_tune_1d5bc_00009,0.75


2025-07-25 19:53:54,172	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/snt/SerraPHIM/data/ray_tune_results/xgb_receptor_tune' in 0.0095s.
2025-07-25 19:53:54,183	INFO tune.py:1041 -- Total run time: 92.58 seconds (92.37 seconds for the tuning loop).


In [ ]:
best_config = analysis.get_best_config(metric="auc", mode="max")
print("Best hyperparameters:", best_config)

# Train final model using best config
best_model = XGBClassifier(
    **best_config,
    eval_metric='logloss',
    use_label_encoder=False,
    n_jobs=8
)
best_model.fit(X_full, y_full)
best_model.save_model(f"{results_path}/xgb_receptor_model_tuned{data_suffix}.json")

Best hyperparameters: {'learning_rate': 0.006838535213235038, 'max_depth': 5, 'n_estimators': 100, 'scale_pos_weight': np.float64(0.7777777777777777), 'subsample': 0.9108073952348492, 'colsample_bytree': 0.751917908204838}


/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:00:46] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [50]:
# GROUP STRATEGY — choose one of: 'receptor', 'phage', or 'both'
group_by = 'receptor'

if group_by == 'receptor':
    groups = np.array(groups_receptors)
elif group_by == 'phage':
    groups = np.array(groups_phage)
elif group_by == 'both':
    # Optional: define each group as a (phage, receptor) pair
    groups = np.array([f"{b}_{p}" for b, p in zip(groups_receptors, groups_phage)])
else:
    raise ValueError("Invalid grouping strategy selected.")

print(f"Number of unique groups: {len(set(groups))}")

Number of unique groups: 5


In [53]:
# Convert group names to numeric codes (required for splitting)
unique_groups = list(set(groups))
group_to_id = {g: i for i, g in enumerate(unique_groups)}
group_ids = np.array([group_to_id[g] for g in groups])

k = 5
gkf = GroupKFold(n_splits=k)
results = []
true_labels = []

for train_idx, test_idx in tqdm(gkf.split(X_full, y_full, groups=group_ids), total=k):
    X_train, X_test = X_full[train_idx], X_full[test_idx]
    y_train, y_test = y_full[train_idx], y_full[test_idx]

    imbalance = np.sum(y_train == 1) / np.sum(y_train == 0)
    best_config["scale_pos_weight"] = 1 / imbalance
    
    xgb = XGBClassifier(
        **best_config,
        n_jobs=8,
        eval_metric='logloss',
        use_label_encoder=False
    )
    xgb.fit(X_train, y_train)
    y_score = xgb.predict_proba(X_test)[:, 1]

    results.append(y_score)
    true_labels.append(y_test)

# Save results
with open(f"{results_path}/serr_receptor_level_kfold_results.pickle", "wb") as f:
    pickle.dump({'scores': results, 'labels': true_labels}, f)

  0%|          | 0/5 [00:00<?, ?it/s]/home/snt/.pyenv/versions/yakuza_venv/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [20:14:31] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
100%|██████████| 5/5 [00:00<00:00, 14.63it/s]


In [55]:
results, true_labels

([array([0.63587373, 0.36634678, 0.63587373, 0.36634678], dtype=float32),
  array([0.50473005, 0.39629772, 0.6089619 , 0.49906892], dtype=float32),
  array([0.50473005, 0.39629772, 0.6089619 , 0.49906892], dtype=float32),
  array([0.50263625, 0.39459795, 0.60965633, 0.5001147 ], dtype=float32),
  array([0.63587373, 0.36634678, 0.63587373, 0.36634678], dtype=float32)],
 [array([0, 0, 1, 1]),
  array([1, 0, 1, 0]),
  array([1, 0, 1, 0]),
  array([1, 0, 1, 0]),
  array([0, 0, 1, 1])])